<h1><center>Bonds Linear Forecasting</center></h1>
<h2><center>Model Tuning and Feature Engineering</center></h2>

## Objective

In this project you will investigate the optimal set of parameters, features, and their transformations that make a linear forecasting model appropriate for a bond price predictions.

## Data
You will use the [Benchmark Solutions Bond Trade Price Kaggle dataset](https://www.kaggle.com/c/benchmark-bond-trade-price-challenge#description), which can also be found in the NYU shared google drive. The dataset contains nearly a million bond trades and their specifications, such as trade_price, bond coupon, callability, time to maturity, trade reporting delay, trade size, trade type (dealer-dealer, dealer-customer, etc) and curve_based_price. Finally, each trade price also has the last 10 trades preceding it (along with those trade characteristics such as $p_{-i}$, $dt_{-i}$, $\text{volume}_{-i}$, $\text{type}_{-i}$ etc).

The test dataset can be [obtained from the Kaggle Website](https://www.kaggle.com/c/benchmark-bond-trade-price-challenge/data) do not use it until the end of the project.

## Motivation
Bonds have a maturity, which means that their returns are not time homogeneous 
- Indeed, for a *risk-free* zero-coupon bond, as calendar time $t$ approaches maturity $T$, $t\rightarrow{T}$, the price $B_{t,T}\rightarrow{1}$. Therefore it doesn't make sense to model price returns, but instead one bootstraps a risk-free curve $r_{t,T}$ and models instead the curve dynamics at fixed time to maturity $dr(\tau)_t\equiv{dr_{t, t+\tau}}$. One then hopes that the volatility for fixed $\tau$ will be a time-homogeneous function. The complexity of this approach, however, arises from the fact that it is prices that are tradable and not rates. Therefore, the yield curve dynamics will contain drifts, which in turn make the historical estimation a bit more complex. 
- The main advantage of modeling yields, however, is that one can price bonds with diffent coupon streams by discounting each future $t_1-$stream by the discount factor $D_{t,t_1}$ before taking the expectation.
- In the presence of default risk, bond yields can be modeled as a linear combination of a risk-free yield and a credit spread $\lambda_{t}dt$ where $L_{t_1,t_2}=\int^{t_2}_{t_1}dt\lambda_t$ measures the cumulative intensity of default. 
- Furthermore upon default, another variable to model is the recovery rate of the remaining principal which investors typically obtain by liquidating the company. In fact the price dynamics of distressed bonds is many the dynamics of the market's expectation of recovery.

The data you are given has no direct information about the level of the yield $r(\tau)_t$ for a given bond with time to maturity $\tau$ aside from the curve_based_price model-derived price. 
- This also means that you cannot bootstrap a yield curve
- But even if at any given $t$ you had bonds at several expirations $T_i$ for a given company, you would still have to make assumptions about the recovery. And most importantly, many of these bonds are *callable*. This means their price contains various forms of optionality which cannot be inferred without any option price data.
- As a result this data set has massive dimensionality if you don't make any assumptions. You can't even treat bonds of the same company at the same calendar time $t$ to be the same asset.


Yet, the goal of this project is to see how far you can get with linear predictive models and feature engineering. This is an important problem because
- Feature engineering gives you an explicit control over the model choice. In fact, the better the features choice, the more predictive power your model will have.
- In the process, you will discover which features are most relevant for this particular price prediction problem and by how much.

## Methodology and Deliverables

Submit all your code in the 'datasets/all_remaining_projects/bonds_forecast' folder.

### Bond Pricing

Since the actual calendar time for each trade is not known, you canot aggregate various bonds into a strip of bonds from which you can bootstrap a yield curve. Therefore the best you can do is fit a single yield parameter for any given bond. 

There are several models you can use:
- A discrete model: Assuming quarterly coupons and no default, you can fit a single yield parameter for each bond maturing within $\tau=T-t$. Document and implement this pricing model.
- A continuous model: You can also use a continuous approximation where you assume all your remaining coupons are distributed daily until expiration. Document and implement this pricing model.

Make sure that your model implementation is efficient because you will price about one million bonds. You will need to conduct some research as to what are the efficient non-linear solvers in Python. Conduct tests to determine the efficiency of each model and save in a seperate notebook. 

Next, pick the faster of the two models and infer all the bond yields for each trade (past and current) and for each bond. Also compute the bond durations for each trae and bond according to the model. Save the result in yields.pkl file in the bonds_forecast/ folder.

### Feature Engineering

Because you will explore the effect of different features, you will need to create a flexible scikit-learn pipeline which possibly includes several feature transformers as we demonstrated in class. For each bond $i$ you observe $dp_i\equiv\text{trade_price-curve_based_price}

#### Individual Bond Features Benchmark

The [Benchmark Kaggle Website](https://www.kaggle.com/c/benchmark-bond-trade-price-challenge#SubmissionInstructions) already contains a Random Forest Benchmark R script which optimizes the mean absolute error for the trade_price, so you can refer to it for inspiration. However, in this problem, you will be predicting the **relative price discrepancy** which, for a bond $i$ is defined as:
$$
RPD_i = \frac{\text{trade_price}(i)-\text{curve_based_price}(i)}{\text{curve_based_price}(i)}
$$
- Is RPD unbiased in-sample? Or is the curve-based price a biased predictor of the trade_price? 
- How does RPD behave as a function of the different features in your dataset? Can you find which individual bond features are most relevant for predicting $RPD_i$? Conduct an analysis no this.
- Using 10-fold cross-validation, create an OLS benchmark for predicting $RPD_i$ from the features of each individual bond. How does that benchmark change when you include the model yields that you compute in the previous part? How does it change when you remove past trades history? Does it make sense to be regressing on past trade levels or should we regress on past $RPD$ values for each bond? How does your benchmark change when you subtract level-dependent variables from your pipeline?

#### Enhancing The Model With Bond Aggregations

Individual bond information is of limited scope in predicting future price moves. However here you can use the information from other bonds to enhance your model. Here are some suggestions:
- High yield vs low yield bonds could behave differently. Can you investigate the variance and bias of RPD as a function of bond yield? Via bucketing, can you obtain plots of mean $RPD$ as a function of yield?
- Liquid (frequently trading) vs illiquid bonds could behave differently. Investigate the conditional effects on RPD of liquidity both in terms of frequency of trading and trade size.
- Volatile bonds could behave differently. Investigate the conditional effects of past trades volatility on RPD's bias and volatility.
- How do any biases or variance of RPD change if one conditions on dealer-dealer trades only vs dealer-customer or customer-dealer trades? Explain your findings.
- Callability and coupon size can also affect RPD's bias and volatility. Investigate.

Finally, once you've done these exercises, can you do K=20 component TrucatedSVD on your regressors to find the most relevant ones. Is it even possible to do? Make sure [to use the randomized algorithm](http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.TruncatedSVD.html) in Scikit-Learn because the conventional Lapack algorithm will crash when doing SVD on a 1MxNumFeatures matrix! 
- In your list of features, make sure to add not only the individual bond features, but also the Aggregate bond features 
    - E.g. yield_level feature (yield relative to other bonds, is it low, medium, high, extremely high, etc?)
    - liquidity features of various kinds
    - relative price volatility feature (compared to other bonds)
    - etc..
- Use the top few SVD factors to enhance your regression similarly to what we did in class. Do you get a performance improvement? How many factors are you using and why?
- Can you identify the most relevant feature combinations from the factor structure of the top SVD components?

#### Test Set
Once you train your best model, how does it perform on the test set?



### README file and code

Create a README_db.txt file containing:
- The names and emails of all the teammates so you can be contacted by the next user of the dataset
- Description of your code base and how it can be used by others
- Outline who was resposible for which part of the project

Make sure you document your notebooks with explanations so others in the class can use them!